In [2]:
%load_ext autoreload
%autoreload 2

In [25]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [26]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-11 16:00:00+00:00


In [27]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('4h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [28]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-11 11:05:04,125 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 11:05:04,131 INFO: Initializing external client
2025-05-11 11:05:04,131 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 11:05:04,763 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [29]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 


In [30]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,5779,125.0,2025-05-11 12:00:00+00:00
1,5905,53.0,2025-05-11 12:00:00+00:00
2,6072,28.0,2025-05-11 12:00:00+00:00
3,5626,33.0,2025-05-11 12:00:00+00:00
4,5788,73.0,2025-05-11 12:00:00+00:00
5,5788,58.0,2025-05-11 16:00:00+00:00
6,6072,29.0,2025-05-11 16:00:00+00:00
7,5905,45.0,2025-05-11 16:00:00+00:00
8,5779,86.0,2025-05-11 16:00:00+00:00
9,5626,33.0,2025-05-11 16:00:00+00:00


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  10 non-null     int32                  
 1   predicted_demand    10 non-null     float64                
 2   pickup_hour         10 non-null     datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 328.0 bytes


In [32]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-11 11:05:28,147 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 11:05:28,151 INFO: Initializing external client
2025-05-11 11:05:28,151 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 11:05:28,570 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Current UTC time: 2025-05-11 15:05:28.147906+00:00
Next hour: 2025-05-11 16:00:00+00:00
Querying for date range: 2025-05-11 to 2025-05-12
Filtering for hour: 2025-05-11 16:00
Found 5 records


In [17]:
from datetime import datetime, timezone, timedelta
import pandas as pd

def fetch_next_4_hours_from_next_4_multiple():
    # Get current UTC time
    now = datetime.now(timezone.utc)

    # Round to the next hour
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Find the next hour that is a multiple of 4
    hours_to_add = (4 - next_hour.hour % 4) % 4
    next_4_multiple = next_hour + timedelta(hours=hours_to_add)

    # Get current and next date strings
    current_date_str = next_4_multiple.strftime("%Y-%m-%d")
    next_date_str = (next_4_multiple + timedelta(days=1)).strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # Read data for the full day
    df = fg.filter(
        (fg.pickup_hour >= current_date_str) &
        (fg.pickup_hour < next_date_str)
    ).read()

    # Collect predictions for next 4 hours
    predictions = []
    for i in range(4):
        hour_time = next_4_multiple + timedelta(hours=i)
        hour_str = hour_time.strftime("%Y-%m-%d %H:00")
        df_hour = df[df['pickup_hour'] == hour_str]
        print(f"Filtering for hour: {hour_str} | Records found: {len(df_hour)}")
        predictions.append(df_hour)

    result = pd.concat(predictions) if predictions else pd.DataFrame()

    print(f"Current time: {now}")
    print(f"Next 4-multiple hour: {next_4_multiple}")
    print(f"Total records for next 4 hours starting from {next_4_multiple}: {len(result)}")
    return result

# Example usage:
predictions_4h = fetch_next_4_hours_from_next_4_multiple()


2025-05-11 10:33:06,695 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:33:06,703 INFO: Initializing external client
2025-05-11 10:33:06,704 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 10:33:07,206 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 
Filtering for hour: 2025-05-11 16:00 | Records found: 5
Filtering for hour: 2025-05-11 17:00 | Records found: 0
Filtering for hour: 2025-05-11 18:00 | Records found: 0
Filtering for hour: 2025-05-11 19:00 | Records found: 0
Current time: 2025-05-11 14:33:06.695361+00:00
Next 4-multiple hour: 2025-05-11 16:00:00+00:00
Total records for next 4 hours starting from 2025-05-11 16:00:00+00:00: 5


In [18]:
now = datetime.now(timezone.utc)

In [33]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
5,5788,58.0,2025-05-11 16:00:00+00:00
6,6072,29.0,2025-05-11 16:00:00+00:00
7,5905,45.0,2025-05-11 16:00:00+00:00
8,5779,86.0,2025-05-11 16:00:00+00:00
9,5626,33.0,2025-05-11 16:00:00+00:00


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  10 non-null     int32                  
 1   predicted_demand    10 non-null     float64                
 2   pickup_hour         10 non-null     datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 328.0 bytes


In [35]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('4h')

In [36]:
dt = current_date.ceil('4h')


In [37]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-05-11 05:00:00")  
results = query.read()

2025-05-11 11:05:52,706 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 11:05:52,709 INFO: Initializing external client
2025-05-11 11:05:52,709 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 11:05:53,198 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.50s) 


In [38]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.38s) 


,pickup_location_id,predicted_demand,pickup_hour


In [39]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,5779,125.0,2025-05-11 12:00:00+00:00
1,5905,53.0,2025-05-11 12:00:00+00:00
2,6072,28.0,2025-05-11 12:00:00+00:00
3,5626,33.0,2025-05-11 12:00:00+00:00
4,5788,73.0,2025-05-11 12:00:00+00:00
5,5788,58.0,2025-05-11 16:00:00+00:00
6,6072,29.0,2025-05-11 16:00:00+00:00
7,5905,45.0,2025-05-11 16:00:00+00:00
8,5779,86.0,2025-05-11 16:00:00+00:00
9,5626,33.0,2025-05-11 16:00:00+00:00


In [40]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [41]:
predictions = fetch_next_hour_predictions()

2025-05-11 11:06:06,070 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 11:06:06,073 INFO: Initializing external client
2025-05-11 11:06:06,073 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 11:06:06,677 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Current UTC time: 2025-05-11 15:06:06.070267+00:00
Next hour: 2025-05-11 16:00:00+00:00
Found 5 records


In [42]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-11 11:06:10,320 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 11:06:10,323 INFO: Initializing external client
2025-05-11 11:06:10,324 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 11:06:10,836 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [43]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.29s) 


In [44]:
df["pickup_hour"].max()

Timestamp('2025-05-11 16:00:00+0000', tz='Etc/UTC')

In [45]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-11 16:00:00+00:00


In [46]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
5,5788,58.0,2025-05-11 16:00:00+00:00
6,6072,29.0,2025-05-11 16:00:00+00:00
7,5905,45.0,2025-05-11 16:00:00+00:00
8,5779,86.0,2025-05-11 16:00:00+00:00
9,5626,33.0,2025-05-11 16:00:00+00:00
